In [1]:
import mediapipe as mp
import cv2
import numpy as np
import csv
import os
import matplotlib.pyplot as plt
import pickle
import pandas as pd

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [3]:
landmarks = ['class']  # Class is Y - target value
for val in range(1,33+1): # 32 total landmarks
    landmarks+=['x{}'.format(val),'y{}'.format(val),'z{}'.format(val),'v{}'.format(val)]

Deadlift Model

In [4]:
with open('pickle files\deadlift.pkl','rb') as f:
    deadlift_model = pickle.load(f)

Lean Model

In [5]:
with open('pickle files\lean.pkl','rb') as f:
    lean_model = pickle.load(f)

Distance Model

In [6]:
def calculate_distance(a, b):
    return np.sqrt((b[0] - a[0])**2 + (b[1]-a[1])**2)

In [7]:
with open('pickle files\distance.pkl','rb') as f:
    distance_model = pickle.load(f)

In [8]:
# Constants for thresholds
LEAN_CONFIDENCE_THRESHOLD = 0.7
BODYLANG_CONFIDENCE_THRESHOLD_DOWN = 0.7
BODYLANG_CONFIDENCE_THRESHOLD_UP = 0.7
DISTANCE_CONFIDENCE_THRESHOLD = 0.7

Final Code

In [9]:
selected_landmarks = [] #  note in all the databases the landmarks are starting from 1 instead of 0
for i in range(23, 33):
    selected_landmarks += [f'x{i}', f'y{i}', f'z{i}', f'v{i}']

In [10]:
import time
cap = cv2.VideoCapture(0)
current_Stage = ''
current_lean_Status = ''
current_distance_Status = ''
counter = 0 
time.sleep(3)
flag = 1
with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret,frame = cap.read()

        # Recolor Feed
        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make Detections
        results = pose.process(image)
        # Recolor it back
        image.flags.writeable = True
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)

        # Draw Landmarks
        mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_pose.POSE_CONNECTIONS
                                  , mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2))
        
        try:
            # Distance things
            landmarks_mp = results.pose_landmarks.landmark
            left_knee = [landmarks_mp[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks_mp[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            right_knee = [landmarks_mp[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks_mp[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            distance = calculate_distance(left_knee,right_knee)*100

            row = np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
            X = pd.DataFrame([row],columns=landmarks[1:])
            row_dist = row[(23*4):]
            X_dist = pd.DataFrame([row_dist],columns=selected_landmarks)
            bodylang_prob = deadlift_model.predict_proba(X)[0]
            bodylang_class = deadlift_model.predict(X)[0]
            lean_prob = lean_model.predict_proba(X)[0]
            lean_class = lean_model.predict(X)[0]
            distance_prob = distance_model.predict_proba(X_dist)[0]
            distance_class = distance_model.predict(X_dist)[0]
            # Check if the posture is centered with sufficient confidence
            if (lean_class == 1.0 and lean_prob[lean_prob.argmax()] > LEAN_CONFIDENCE_THRESHOLD) or (flag==0) and distance>12 and distance<18:
                flag=0
                # Update the stage based on body language class and confidence
                if bodylang_class == 0.0 and bodylang_prob[bodylang_prob.argmax()] > BODYLANG_CONFIDENCE_THRESHOLD_DOWN:
                    current_Stage = "down"
                elif bodylang_class == 1.0 and current_Stage == "down" and bodylang_prob[bodylang_prob.argmax()] > BODYLANG_CONFIDENCE_THRESHOLD_UP:
                    current_Stage = "up"
                    counter += 1

            # Update distance status
            if distance<11:
                current_distance_Status = "Narrow"
            elif distance>=11 and distance<18:
                current_distance_Status = "Correct"
            else: distance = "Wide"

            # Update lean status
            if lean_class == 0.0 and lean_prob[lean_prob.argmax()] > LEAN_CONFIDENCE_THRESHOLD and current_Stage=="up":
                current_lean_Status = "Left"
            elif lean_class == 1.0 and lean_prob[lean_prob.argmax()] > LEAN_CONFIDENCE_THRESHOLD:
                current_lean_Status = "Centre"
            elif lean_class == 2.0 and lean_prob[lean_prob.argmax()] > LEAN_CONFIDENCE_THRESHOLD and current_Stage=="down":
                current_lean_Status = "Right"
            else: current_lean_Status = "Centre"
            
            # Get status box
            cv2.rectangle(image,(0,0),(550,60),(245,117,16),-1)

            # Display Rep
            cv2.putText(image,'Status',
                        (95,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
            cv2.putText(image,current_Stage,
                            (90,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
            # Display Probability
            cv2.putText(image,'Prob',
                        (15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
            cv2.putText(image,str(round(bodylang_prob[np.argmax(bodylang_prob)],2)),
                        (10,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
            # Display Count
            cv2.putText(image,'Count',
                        (180,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
            cv2.putText(image,str(counter),
                        (200,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
            # Display Lean
            cv2.putText(image,'Posture',
                        (280,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
            cv2.putText(image,current_lean_Status,
                            (280,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
            # Display Distance
            cv2.putText(image,'Distance',
                        (400,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
            cv2.putText(image,current_distance_Status,
                            (400,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
            cv2.imshow('Deadlifts',image)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        except Exception as e:
            print(e)
cap.release()
cv2.destroyAllWindows()
            
            

c:\Users\Yash Phatak\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\Yash Phatak\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Users\Yash Phatak\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Users\Yash Phatak\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatab

'NoneType' object has no attribute 'landmark'


c:\Users\Yash Phatak\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\Yash Phatak\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Users\Yash Phatak\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Users\Yash Phatak\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatab

'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'


c:\Users\Yash Phatak\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\Yash Phatak\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Users\Yash Phatak\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'


c:\Users\Yash Phatak\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\Yash Phatak\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Users\Yash Phatak\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
